# Определение песен на египетском диалекте образом кластеризации

Эта тетрадь решает задачу обучения без учителя текстов песен на арабском языке, и разделяет их на два непересекающихся классов, зависимо от их языковой принадлежности.

Все песни, предусмотрены алгоритму, произведены известной египетской певицой "Умм Кульсум" (на арабском: أمّ كلثوم). Она выступила, и ее песни написала, на стандартном или диалектическом арабском. Полученные песни так входят в один из этих двух языковых групп: песни на стандартном арабском, и песни на египетском диалекте.

Нам не дали меток с языками среди данных задачи. Употребляние алгоритмов классификации с учителем так невозможно.

Поэтому решаем задачу как следует: используя алгоритмы векторизации и кластеризации, определяем разделение между теми двуми классами языков на основе максимизации разницы признаков самых же классов. Оценка результатов расчета, получена от автоматического оценщика Kaggle, является {'Public':1.0, 'Private':0.85714}. Впоследствии этого, можно безусловно утверждать, что алгоритм хорошо работает на рашение задачи этого чалленджа.

Полное описание задачи здесь: https://www.kaggle.com/c/om-kalthom-tiny/

# Подключение библиотек

### Основые модули и постоянные величины

Как обычно начинаем, настраивая все необходимые библиотеки для предварительной обработки данных, и определяя константы, употребляемые в ходе скрипта.

In [1]:
import numpy as np # Модуль для вычисления с векторами в линейной алгебре
np.random.seed(42) # Установление генератор случайности

import pandas as pd # Основая библиотека для загрузки баз данных
import os # Содержит функции для работы с операционной системой 
import nltk # Специализированная библиотека для обработки естественного языка

DIR = './txts/' # Директория, где находятся тексты всех нашей песен, относительно к местонахождению тетради
CLUSTERS = ['arabic_standard','egyptian_dialect'] # Сколько языковых кластеров

### Внимание: наименование файлов

Те операционные системы, у которых не установлен набор шрифтов арабских символов, не хорошо обработают с именами файлов, содержащими те символы. Файлы получены от задачи в Kaggle все имеют имена с арабскими символами. Целесообразно так запустить следующую ячейку, если нужно Вашей операционной системе: в том случае, меняйте тип ячейки на "Код" и запускайте.

# Построение синтактического анализатора

Чтобы создать датафрейм с чистыми даннами, необходимо прежде всего узнать, как представляются сырые данные задачи, и подумать как хотели бы мы их очистить, чтобы последовательно векторизатор принял их успешно.

Показываем так один пример полученных файлов, чтобы объяснить типичную структуру множества тех файлов.

Важно здесь подчеркнуть, как процесс выявления оснвной структуры данных, называемый по-английски *Exploratory Data Analysis* (разведочный анализ данных), является наиболее важным моментом целого процесса кластеризации текстов. Алгоритмы машинного обучения не могут правильно извлечь признаки данных, если самые же данные не получены одинаково форматтированы. В худшем случае, если алгоритмы получают данные различного типа (например *int* совместно со *string*) они совсем не работают. 

In [2]:
first_file = DIR+os.listdir(DIR)[0]
with open(first_file,'r',encoding='utf-8') as f:
    print(f.read()[:200])
print('\nDone')

title:هذه ليلتي
poet: جورج جرداق
composer: محمد عبدالوهاب
year: ١٩٦٨

   هذه ليلتي وحلم حياتى

   بين ماض من الزمان وآت

   الهوى أنت كله والأمانى

   فأملأ الكأس بالغرام و هات

   بعد حين يبدل الحب د

Done


### Рассмотрение примера файла

Первым файлом является песня هذه ليلتي (*хазихи лайлатии*, "эта моя ночь"), и как видно в тексте выше, слова песни предшествуют шапку с метаданными, содержащую название песни, поэта, композитора, год опубликования. 

Наш синктактический анализатор должен так извлечь подряд следующие данные из файлов с сырыми текстами:

- наименование песни,
- имя поэта,
- имя композитора,
- год опубликования,
- сам текст песни
    
После извлечения тех данных, мы хотели бы включить их в единый датафрейм *pandas*, который нам поможет в визуализации полученной информации и в ее предварительной обработке.

Основная структура желательного датафрейма - следующая. 

Важно помнить о том, что словари в Python - неупорядоченны обекты; итак столбцы датафреймов pandas, созданных словарями, также неупорядоченны.

In [3]:
df_example = pd.DataFrame({'file_name':'song.txt',
                           'title':'title_of_song',
                           'poet':'poet_name',
                           'composer':'composer_name',
                          'year':2000,
                          'text':['long string with all rows merged together']})
df_example 

,composer,file_name,poet,text,title,year
0,composer_name,song.txt,poet_name,long string with all rows merged together,title_of_song,2000


### Определение задачи синтактического анализатора

Синтактический анализатор должен выполнять подряд следующие задачи:

- Открыть каждый файл;

- Прочитать первые 4 строки. Извлечь из каждой строки, следовательно, наименование песни, поэт, композитор, и год;

- Прочитать осталные строки. Извлечь из них ненулевые строки, и объединить их в единую строку

- Наконец, создать датафрейм с полученной информацией.

Вот тут код такого же анализатора с комментированией о него функциях.

In [4]:
# Удаются пустые списки строки
filenames, titles, poets, composers, years, texts = [], [], [], [], [], [] 

# Вызываем функцию os.listdir(), которая возвращает список всех файлов для последовательной итерации над ними
files_in_directory = os.listdir(DIR) 

counter_files = 0 # Сколько файлов посмотрены синтактическим анализатором. Инициализируется на 0.
    
for filename in files_in_directory:                           # Основная итерация
    
    with open(DIR+filename,'r', encoding='utf-8') as f:       # Менедфер контекста. Более удобный чем try/except
        
        filenames.append(filename)
        
        title = f.readline().strip('\n').split(':')           # Для каждой из превых 4 строк, извлекает подстроку,
        titles.append(title[1])                               # имеющую искаемую информацию. Работает так:
        poet = f.readline().strip('\n').split(':')            # f.readline() возвращает одну строку из файла;
        poets.append(poet[1])                                 # string.strip('\n') удаляет симболы с оьратным слэшем;
        composer = f.readline().strip('\n').split(':')        # и string.split(символ) возвращает кортеж элементов 
        composers.append(composer[1])                         # строки, разделена переданным символом.
        year = f.readline().strip('\n').split(':')            # Интересующий нас элемент находится на втором месте кортежа,
        years.append(year[1])                                 # с индексом 1
        
        text = ''.join(f.read().replace('\n',''))             # Читает все остальное, и возращает одну ненулевую,
        texts.append(text)                                    # объединенную строку.
        
        counter_files+=1
    
dictionary_of_songs = {'file_name':filenames,                 # Создание словаря с извелченными даннами
                       'title':titles,
                       'poet':poets,
                       'composer':composers,
                       'year':years,
                       'text':texts}

# Проверяем, что все файлы из директории были правильно проанализированы
print('{} file(s) parsed out of {} file(s) in the directory.'.format(counter_files, len(files_in_directory)))
if counter_files==len(files_in_directory):
    print('All files were analysed correctly') 
else:
    raise AssertionError('Some files were not analysed')

20 file(s) parsed out of 20 file(s) in the directory.
All files were analysed correctly


# Создание объекта "pandas.DataFrame"

Используя словарь с извлеченными даннами, можно теперь создать объект DataFrame, который будет содержать данные о песнях, и над которым осуществляем требующий процесс предварительной обработки.

Почему употребляется здесь назвние "pre_merge", значит "до объединения", в качестве наименования созданного объекта? Посмотрим позже, что случится во время объединиения этого датафрейма с датафреймом, содержащим индексы песен: тогда все будет яснее. 

In [5]:
df_pre_merge = pd.DataFrame(dictionary_of_songs, columns=['file_name','title','poet','composer','year','text'])
print(df_pre_merge.shape)

(20, 6)


За того, что мы получили только 20 песен, можно здесь показать весь датафрейм

In [6]:
# Следующая строка показывает все строки датафрейма, до той строки, 
# индекс которой является последным элементом самого датафрейма.
# Так представляются все строки датафрейма подряд.

df_pre_merge.head(df_pre_merge.shape[0]) 

,file_name,title,poet,composer,year,text
0,song_0.txt,هذه ليلتي,جورج جرداق,محمد عبدالوهاب,١٩٦٨,هذه ليلتي وحلم حياتى بين ماض من الزمان وآ...
1,song_1.txt,هجرتك,أحمد رامي,رياض السنباطي,١٩٥٩,هجرتك يمكن أنسى هواكو أودع قلبك القاسيو قلت أق...
2,song_10.txt,إنت عمري,أحمد شفيق كامل,محمد عبدالوهاب,١٩٦٤,رجعوني عنيك لأيامي اللي راحواعلموني أندم على ا...
3,song_11.txt,فات الميعاد,مرسي جميل عزيز,بليغ حمدي,١٩٦٧,فات الميعاد و بقينا بعاد بعادوالنار بقت دخان ...
4,song_12.txt,ظلمنا الحب,عبدالوهاب محمد,بليغ حمدي,١٩٦٢,أنا و إنت ظلمنا الحبظلمنا الحب بإيديناو جنينا ...
5,song_13.txt,عودت عيني,أحمد رامي,رياض السنباطي,١٩٥٧,عودت عيني على رؤياكو قلبي سلم لك أمريأشوف هنا ...
6,song_14.txt,أروح لمين,عبدالمنعم السباعي,رياض السنباطي,١٩٥٨,أروح لمينو أقول يا مين ينصفني منكما هو إنت فرح...
7,song_15.txt,أغدا ألقاك,الهادي آدم,محمد عبدالوهاب,١٩٧١,أغدا ألقاكيا خوف فؤادي من غديالشوقي و إحتراقي ...
8,song_16.txt,للصبر حدود,عبدالوهاب محمد,محمد الموجي,١٩٦٤,ماتصبرنيش بوعود و كلام معسول و عهودأنا ياما صب...
9,song_17.txt,أنساك,مأمون الشناوي,بليغ حمدي,١٩٦٢,أنساك ده كلامانساك ياسلامأهو ده اللي مش ممكن أ...


### Загрузка индексов

Загружаем теперь файл id.csv с индексами всех песен, которые используются на приготовление файла с окончательными решениями про кластеризации текстов.

In [7]:
ids = pd.read_csv('id.csv')
print(ids.shape)
ids.head(ids.shape[0])

(20, 2)


,id,title
0,1,سيرة الحب
1,2,الأطلال
2,3,فكروني
3,4,إنت عمري
4,5,إسأل روحك
5,6,حب إيه
6,7,أغدا ألقاك
7,8,حيرت قلبي معاك
8,9,هذه ليلتي
9,10,دارت الأيام


Проверяем, что сколько песен просмотрели анализатором, столько условных обозначений поучили.

In [8]:
assert (df_pre_merge.shape[0]==ids.shape[0]), r"Attention! Number of songs different from number of Id's"

# Объединение датафреймов

Попробуем осуществлять объединения датафреймов "df_pre_merge" и "ids", чтобы добавить первому все обозначения второго.

In [9]:
df_post_merge = pd.merge(df_pre_merge,ids, on='title')
print(df_post_merge.shape)
print('\nDo they have the same size? ', df_post_merge.shape[0]==df_pre_merge.shape[0])

(19, 7)

Do they have the same size?  False


# Внимание, внимание!

Кажется, что что-то случилось в течение процесса объединения, и размер объекта в его результате отличается от размера оригинальных объектов на единицу. 

Посмотрим так с помощью функции конкатенации .concat(), какая строка первого объекта отсутствует во втором.

In [10]:
df_concatenated = pd.concat([df_pre_merge.set_index('title'), df_post_merge.set_index('title')], 
                   axis='columns', keys=['df_pre_merge', 'df_post_merge'])
df_concatenated

df_pre_merge                                                \
                   file_name                poet         composer     year   
 سيرة الحب        song_3.txt      مرسي جميل عزيز        بليغ حمدي     ١٩٦٤   
أروح لمين        song_14.txt   عبدالمنعم السباعي    رياض السنباطي     ١٩٥٨   
أغدا ألقاك       song_15.txt          الهادي آدم   محمد عبدالوهاب     ١٩٧١   
ألف ليله و ليله  song_18.txt      مرسي جميل عزيز        بليغ حمدي     ١٩٦٩   
أنساك            song_17.txt       مأمون الشناوي        بليغ حمدي     ١٩٦٢   
إسأل روحك         song_8.txt     عبد الوهاب محمد      محمد الموجي     ١٩٧٠   
إنت عمري         song_10.txt      أحمد شفيق كامل   محمد عبدالوهاب     ١٩٦٤   
الأطلال           song_4.txt        إبراهيم ناجي    رياض السنباطي     ١٩٦٦   
حب إيه            song_6.txt      عبدالوهاب محمد        بليغ حمدي     ١٩٦٠   
حكم علينا الهوى   song_7.txt      عبدالوهاب محمد        بليغ حمدي     1973   
حيرت قلبي معاك    song_5.txt           أحمد رامي    رياض السنباطي     ١٩٦١   
دارت الأيام       song_9.txt       مأمون الشناوي   محمد عبدالوهاب     ١٩٧٠   
ظلمنا الحب       song_12.txt      عبدالوهاب محمد        بليغ حمدي   ١٩٦٢     
عودت عيني        song_13.txt           أحمد رامي    رياض السنباطي     ١٩٥٧   
فات الميعاد      song_11.txt      مرسي جميل عزيز        بليغ حمدي     ١٩٦٧   
فكروني           song_19.txt      عبدالوهاب محمد   محمد عبدالوهاب     ١٩٦٦   
للصبر حدود       song_16.txt      عبدالوهاب محمد      محمد الموجي     ١٩٦٤   
هجرتك             song_1.txt           أحمد رامي    رياض السنباطي     ١٩٥٩   
هذه ليلتي         song_0.txt          جورج جرداق   محمد عبدالوهاب     ١٩٦٨   
يا مسهرني         song_2.txt           أحمد رامي        سيد مكاوي     ١٩٧٢   

                                                                    \
                                                              text   
 سيرة الحب       طول عمري بأخاف من الحبوسيرة الحبوظلم الحب لكل ...   
أروح لمين        أروح لمينو أقول يا مين ينصفني منكما هو إنت فرح...   
أغدا ألقاك       أغدا ألقاكيا خوف فؤادي من غديالشوقي و إحتراقي ...   
ألف ليله و ليله  يا حبيبيالليل و سماه و نجومه و قمرهقمره و سهره...   
أنساك            أنساك ده كلامانساك ياسلامأهو ده اللي مش ممكن أ...   
إسأل روحك        إسأل روحك إسأل قلبكقبل ما تسأل إيه غيرنيأنا غي...   
إنت عمري         رجعوني عنيك لأيامي اللي راحواعلموني أندم على ا...   
الأطلال          يا فؤادي لا تسل أين الهوىكان صرحا من خيال فهوى...   
حب إيه           حـب إيه اللي إنت جاي تقول عليــهإنت عارف قبله ...   
حكم علينا الهوى  حكم علينا الهوى نعشق سوا يا عينو إحنا اللي قبل...   
حيرت قلبي معاك   حيرت قلبي معاكو أنا بداري وأخبيقولي أعمل ايه و...   
دارت الأيام      و دارت الأيام و مرت الأيامما بين بعاد و خصامو ...   
ظلمنا الحب       أنا و إنت ظلمنا الحبظلمنا الحب بإيديناو جنينا ...   
عودت عيني        عودت عيني على رؤياكو قلبي سلم لك أمريأشوف هنا ...   
فات الميعاد       فات الميعاد و بقينا بعاد بعادوالنار بقت دخان ...   
فكروني           كلموني تاني عنكفكروني    فكرونيصحوا نار الشوق ...   
للصبر حدود       ماتصبرنيش بوعود و كلام معسول و عهودأنا ياما صب...   
هجرتك            هجرتك يمكن أنسى هواكو أودع قلبك القاسيو قلت أق...   
هذه ليلتي           هذه ليلتي وحلم حياتى   بين ماض من الزمان وآ...   
يا مسهرني        ماخطرتش على بالك يومتسأل عنيو عنيه مجافيها الن...   

                df_post_merge                                                \
                    file_name                poet         composer     year   
 سيرة الحب                NaN                 NaN              NaN      NaN   
أروح لمين         song_14.txt   عبدالمنعم السباعي    رياض السنباطي     ١٩٥٨   
أغدا ألقاك        song_15.txt          الهادي آدم   محمد عبدالوهاب     ١٩٧١   
ألف ليله و ليله   song_18.txt      مرسي جميل عزيز        بليغ حمدي     ١٩٦٩   
أنساك             song_17.txt       مأمون الشناوي        بليغ حمدي     ١٩٦٢   
إسأل روحك          song_8.txt     عبد الوهاب محمد      محمد الموجي     ١٩٧٠   
إنت عمري          song_10.txt      أحمد شفيق كامل   محمد عبدالوهاب     ١٩٦٤   
الأطلال            song_4.txt        إبراهيم نا

Даже если на первый взгляд датафрейм представляется полным, нам необходимо отметить следующий:

Песня 'song_3.txt', سيرة الحب (*сира-т-ул-хубб*, "песня любви") отсутствует в датафрейме, являющихся итогом процесса объединения. Посмотрим как ее пойск в объедененном датафрайме возвращает нулевой объект.

In [11]:
print(df_post_merge.loc[df_post_merge['file_name']=='song_3.txt'])
df_post_merge.head(df_post_merge.shape[0])

Empty DataFrame
Columns: [file_name, title, poet, composer, year, text, id]
Index: []


,file_name,title,poet,composer,year,text,id
0,song_0.txt,هذه ليلتي,جورج جرداق,محمد عبدالوهاب,١٩٦٨,هذه ليلتي وحلم حياتى بين ماض من الزمان وآ...,9
1,song_1.txt,هجرتك,أحمد رامي,رياض السنباطي,١٩٥٩,هجرتك يمكن أنسى هواكو أودع قلبك القاسيو قلت أق...,11
2,song_10.txt,إنت عمري,أحمد شفيق كامل,محمد عبدالوهاب,١٩٦٤,رجعوني عنيك لأيامي اللي راحواعلموني أندم على ا...,4
3,song_11.txt,فات الميعاد,مرسي جميل عزيز,بليغ حمدي,١٩٦٧,فات الميعاد و بقينا بعاد بعادوالنار بقت دخان ...,18
4,song_12.txt,ظلمنا الحب,عبدالوهاب محمد,بليغ حمدي,١٩٦٢,أنا و إنت ظلمنا الحبظلمنا الحب بإيديناو جنينا ...,16
5,song_13.txt,عودت عيني,أحمد رامي,رياض السنباطي,١٩٥٧,عودت عيني على رؤياكو قلبي سلم لك أمريأشوف هنا ...,17
6,song_14.txt,أروح لمين,عبدالمنعم السباعي,رياض السنباطي,١٩٥٨,أروح لمينو أقول يا مين ينصفني منكما هو إنت فرح...,14
7,song_15.txt,أغدا ألقاك,الهادي آدم,محمد عبدالوهاب,١٩٧١,أغدا ألقاكيا خوف فؤادي من غديالشوقي و إحتراقي ...,7
8,song_16.txt,للصبر حدود,عبدالوهاب محمد,محمد الموجي,١٩٦٤,ماتصبرنيش بوعود و كلام معسول و عهودأنا ياما صب...,19
9,song_17.txt,أنساك,مأمون الشناوي,بليغ حمدي,١٩٦٢,أنساك ده كلامانساك ياسلامأهو ده اللي مش ممكن أ...,15


Давайте рассмотрим, почему объединение на название файла 'song_3.txt' не запускалось правильно.

In [12]:
title_from_id = ids[ids['title']=='سيرة الحب']['title'].values
print(title_from_id, len(title_from_id[0]))

title_from_df = df_pre_merge[df_pre_merge['file_name']=='song_3.txt']['title'].values
print(title_from_df, len(title_from_df[0]))

['سيرة الحب'] 9
[' سيرة الحب'] 10


Как видно, имя песни, найденное в файле, отличается от названия песни, содержанного в файле с индексами. Точнее, продолжает оставать нежеланную пробель в строке с названием, выходящей с синктактического анализатора.

Это очень хороший пример, благодоря которого подчеркивается важность распостраненного контроля внутренной согласованности наших данных, по всем шагам процесса их обработки.

Добавляем так вручную отсутствущю информацию про песне 'song_3.txt', про песне любви.

In [13]:
df_post_merge = df_post_merge.append(df_pre_merge[df_pre_merge['file_name']=='song_3.txt'])

mask = df_post_merge.loc[:,('file_name')] == 'song_3.txt'
df_post_merge.loc[mask,('id')] = 1

df_post_merge['id'] = df_post_merge['id'].astype(int)
print(df_post_merge.shape)
df_post_merge.head(df_post_merge.shape[0])

(20, 7)


,composer,file_name,id,poet,text,title,year
0,محمد عبدالوهاب,song_0.txt,9,جورج جرداق,هذه ليلتي وحلم حياتى بين ماض من الزمان وآ...,هذه ليلتي,١٩٦٨
1,رياض السنباطي,song_1.txt,11,أحمد رامي,هجرتك يمكن أنسى هواكو أودع قلبك القاسيو قلت أق...,هجرتك,١٩٥٩
2,محمد عبدالوهاب,song_10.txt,4,أحمد شفيق كامل,رجعوني عنيك لأيامي اللي راحواعلموني أندم على ا...,إنت عمري,١٩٦٤
3,بليغ حمدي,song_11.txt,18,مرسي جميل عزيز,فات الميعاد و بقينا بعاد بعادوالنار بقت دخان ...,فات الميعاد,١٩٦٧
4,بليغ حمدي,song_12.txt,16,عبدالوهاب محمد,أنا و إنت ظلمنا الحبظلمنا الحب بإيديناو جنينا ...,ظلمنا الحب,١٩٦٢
5,رياض السنباطي,song_13.txt,17,أحمد رامي,عودت عيني على رؤياكو قلبي سلم لك أمريأشوف هنا ...,عودت عيني,١٩٥٧
6,رياض السنباطي,song_14.txt,14,عبدالمنعم السباعي,أروح لمينو أقول يا مين ينصفني منكما هو إنت فرح...,أروح لمين,١٩٥٨
7,محمد عبدالوهاب,song_15.txt,7,الهادي آدم,أغدا ألقاكيا خوف فؤادي من غديالشوقي و إحتراقي ...,أغدا ألقاك,١٩٧١
8,محمد الموجي,song_16.txt,19,عبدالوهاب محمد,ماتصبرنيش بوعود و كلام معسول و عهودأنا ياما صب...,للصبر حدود,١٩٦٤
9,بليغ حمدي,song_17.txt,15,مأمون الشناوي,أنساك ده كلامانساك ياسلامأهو ده اللي مش ممكن أ...,أنساك,١٩٦٢


In [14]:
assert df_post_merge.shape[0]==df_pre_merge.shape[0]

Сейчас все песни правлиьно входят в соответствующую их строку, и все строки имеют полную информацию про песням.

Меняем метку датафрейма, для нашего удобства: кроме этого, предварительная обработка данных наконец закончилась!

In [15]:
df = df_post_merge

### Сохранения датафрейма (не обязательно)

Наверное Вы не хотите передать данные нашей модели, прямо от оперативной памяти и без предыдущего сохранения самых, боясь аварийный отказ операционной системы и последующую потерю информации. Молодцы, вы не одиноки. 

Выполняйте следующие две ячейки кода для сохранения и/или загрузки датафрейма на жетский диск, чтобы больше не волноваться.

# Загрузка моделей машинного обучения

Установливаем cейчас все нужны конструкторы машинного обучения. Задача кластеризации текстов осуществляется этим образом^

1) Во-первых, переобразуем строки текстов на *tokens* слов, и включаем те слова в словарь с индексами признаков. Эта задача решается образом *векторизатора*.

2) Во-вторых, обучим *кластеризатор* какого-то рода с матрицой признаков, выходящей от векторизатора.

3) Во-третьих, повтаряем предыдущие пункты для каждого из испытательных гиперпараметров конструкторов.

Поэтому нам будет нужно установливать и создать следующие конструкторы:

- Один векторизатор;
- Один кластеризатор;
- Один пайплайн;
- Один объект для пойска сетки.

Много опций доступны, мы выбираем следующие:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer # Векторизатор
from sklearn.cluster import KMeans                          # Кластеризатор
from sklearn.pipeline import Pipeline                       # Пайплайн
from sklearn.model_selection import GridSearchCV            # Объект для пойска сетки

### Stopwords

Установливаем также список шумовых слов, которые наш векторизатор не включит в словарь индексов признаков.

In [17]:
stopwords = nltk.corpus.stopwords.words('arabic')
print('How many stopwords: {}'.format(len(stopwords)))
stopwords[:10] # Показываем 10 примеров загруженных шумовых слов

How many stopwords: 248


['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي']

Определяем независимую переменную, которая входит в качестве инпута в нашу модель.

In [18]:
COLUMNS = 'text'
X = df[COLUMNS]
print('{} song(s) as an input.'.format(len(X)))

20 song(s) as an input.


# Обучение моделей

Создаем конструкторы векторизатора и кластеризатора, и установливаем пайплайн для пойска светки их оптимизированных гиперпараметров.

По поводу векторизатора, нам интересно узнать, сколько ngrams слов передать кластеризатору, и если кластеризатор работает лучше со словами, взвешеннами обратной частотой документов, или невзвешеннами том образом.

В противоположность векторизатору, нам не нужно искать наиболее еффективные параметры для кластеризатора. 
Это следует от того, что самый важный гиперпараметер для кластеризатора KMeans фактически является n_clusters, число кластеров, которое нам дали в постановке задачи. Действительно, тексты обязательно разделены на две группы, и нам это будет достаточным.

Пойск светки гиперпараметров моделей в соответствии с предыдущими рекомендациями осуществляется этим образом:

In [19]:
n_clusters = len(CLUSTERS) # Сколько найдутся языковых кластеров

pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words=stopwords, # Указываем список шумовых слов
                                             )),
                    ('cluster',KMeans(n_clusters=n_clusters,
                                     random_state=42
                                     ))
                    ])

params = {'tfidf__use_idf':(True,False),                  # Употребляется ли взвешивание обратной частотой документов
          'tfidf__ngram_range':((1,1),(1,2),(2,2),        # На сколько н_граммы сгруппировать слова
                                (1,3),(2,3),(3,3),
                                (1,4),(2,4),(3,4),(4,4)
                               )
         }

gridsearch = GridSearchCV(pipeline,
                          params,
                          verbose=1,
                          n_jobs=-1)

gridsearch.fit(X)

print(gridsearch.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   20.3s finished


{'tfidf__ngram_range': (3, 4), 'tfidf__use_idf': False}


# Постановка прогнозов

Самый эффективный оценщик, найден алгоритмом пойска светки, является векторизатор, который не взвешивает слова образом их обратной частоты в документах, и который объединяет слова на множества трех и четырех элементов.

Вызываем метод .predict() самого эффективного оценщика, который останавливается в атрибуте .best\_estimator\_ консруктора gridsearch после его обучения. С помощью этого метода формулируем список прогнозов о кластерной принадлежности различных текстов.

In [20]:
predictions = gridsearch.best_estimator_.predict(X)
print(predictions)
dialect = predictions

# Невозможно априорно предсказать, какую метку получат песни на диалекте, и какую метку песни на стандартном арабском.
# Обе категории могут предположить величину 0 или 1 в результате процесса кластеризации.
# Если оценка Kaggle показывает, что определили классы наоборот, надо выполнять и следующий код:

# invert_predictions = (predictions==0).astype(int)
# print(invert_predictions)
# dialect = invert_predictions

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


Соэдаем датафрейм с прогнозами для отправления на Kaggle, и проверяем что все случилось без каких-либо проблем.

In [21]:
predictions_df = pd.DataFrame({'dialect':dialect},index=df_post_merge['id'])

# Система оценки Kaggle ожидает файл с этими названиами для столбцов: ['ID','dialect'].
# Нужно так менять название столбца с индексами.
predictions_df.index.name = 'ID' 

print(predictions_df.shape)
predictions_df.head(5)

(20, 1)


,dialect
ID,
9,0
11,0
4,0
18,0
16,0


Рассмотрим, какие песни определились как песни диалекта.

In [22]:
predictions_df[predictions_df['dialect']==1]

,dialect
ID,
2,1


Есть только одна.

Кажется, что наш кластеризатор определяет одну и только одну песню, текст которой значительно отличается от остальных. 

Показываем, для нашего любопытства, доступную информацию о той необычной песне.

In [23]:
element_to_find = predictions_df[predictions_df['dialect']==1].index[0] # Извлекам индекс интересующего элемента
df[df['id'] == element_to_find]                                         # Смотрим на элемент с извлеченным индексом

,composer,file_name,id,poet,text,title,year
13,رياض السنباطي,song_4.txt,2,إبراهيم ناجي,يا فؤادي لا تسل أين الهوىكان صرحا من خيال فهوى...,الأطلال,١٩٦٦


Вот она одна песня, الأطلال (*ал-атлаал*, "руины"), текст которой, по нашему алгоритму, не написанном на стандартном арабском языке.

# Запись прогнозов и их оценка

Выполнив все требования задачи, записываем теперь форматтированный файл с решениями, и отправляем его на Kaggle.

Полученная оценка является {'Public':1.0, 'Private':0.85714}. Это значит, что все песни публичной таблицы лидеров праивльно предсказали, и тоже 85% песен непубличной таблицы. 

Вероятно есть еще одна песня, которая принадлежит группе диалетских песен, и которую мы неправильно определили как песня стандартного арабского языка. Так или иначе, полученная оценка является очень высока, для такой простой модели.

In [24]:
predictions_df.to_csv('submission.csv')
print('Done')

Done


# Остальные аспекты, имеющие интерес по лингвистике

Даже если уже закончили обязательную нам работу, было бы интересно узнать, как модель определила лингвистический кластер предусмотренных песен, значит, на основе каких признаков. 

С этой целью вызываем адаптированный код образца, находящегося в документации scikit-learn: http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

Во-первых показывем описание нашего лучшего оценщика:

In [25]:
gridsearch.best_estimator_.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(3, 4), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما',..., 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن', 'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=False,
          vocabulary=None)),
 ('cluster',
  KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
      n_clusters=2, n_init=10, n_jobs=1, pre

Давайте разделяем оценщик на его основные компоненты, один векторизатор и один кластеризатор.

In [26]:
best_vectorizer = gridsearch.best_estimator_.steps[0][1] # Определяем векторизатор

best_clusterer = gridsearch.best_estimator_.steps[1][1]  # Определяем кластеризатор

Употребляем атрибут *.cluster\_centers\_* лучшего кластеризатора для определения *н\_граммов*, наиболее характеризующих каждый кластер.

In [27]:
order_centroids = best_clusterer.cluster_centers_.argsort()[:, ::-1] # Условное обозначение всех н_граммов, от наиболее  
                                                                     # частых до самых редких
print(order_centroids.shape)

(2, 5637)


Количество строк равно числу кластеров, а количество столбцов - числу н\_граммов.

Массив определяет 5637-мерное векторное пространство, но нам доставят первые 10 н\_граммов.

In [28]:
print(order_centroids[:,:10])

[[2761 1428 2760 4659 5604 4660   21   20 3692 3691]
 [4186 3337 2222 2221 4988 4987 2212 2211 2210 2209]]


Для каждого из кластеров извлекаем и распечатывем н\_граммы, характеризующие наиболее высокой частатой.

In [29]:
terms = best_vectorizer.get_feature_names()
for i in range(len(CLUSTERS)):
    print("Cluster {}:\n".format(CLUSTERS[i]))
    for ind in order_centroids[i, :10]:
        print(terms[ind])
    print()

Cluster arabic_standard:

ده اللي مش ممكن
اللي مش ممكن
ده اللي مش
مر يوم رؤياكما
يوم رؤياكما ينحسبش
مر يوم رؤياكما ينحسبش
أبداأهو ده اللي مش
أبداأهو ده اللي
غيرك أبداأهو ده اللي
غيرك أبداأهو ده

Cluster egyptian_dialect:

كالحريقو الدنيا نعرفهاو
عدونا فسبقنا ظلناو
تنسىو تعلم تمحويا حبيبي
تنسىو تعلم تمحويا
نحويمن خلال الموج مدت
نحويمن خلال الموج
تمحويا حبيبي شيئ بقضاءما
تمحويا حبيبي شيئ
تمتد نحويمن خلال الموج
تمتد نحويمن خلال



# Краткое лингвистическое замечание об итогах обучения

Интересно отметить, как выражение اللي مش ممكن, (*илли миш мумкин*, "который невозможно"), которое является типичным выражением диалекта, было определено оценщиком как предсказатель высокой надежности кластера "стандартного арабского", а не диалекта. Это же выражение, или часть его, фактически представляется в первом, втором, третьем, седьмом, восьмом и девятом месте среди первых десяти предсказателей! Но на самом деле, вариант выражения на стандартном арабском представлялось бы как "الذي لا يمكن", *аллази ла юмкин*. Кажется, что алгоритм выявляет неожиданные признаки обычных текстов.

Это пример показывает как данные, собранные в реальном мире, обычно имеют особые характеристики, которые отличаются от ожиданных характеристиков абстрактных моделей. Поэтому пусть не представляется странным, что тексты правильно определены как тексты на стандартном арабском, содержат слова диалекта, и содержат много тех слов.